In [17]:
import pandas as pd
import catboost
import pickle
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from numba import jit
import numpy as np

In [29]:
train = pd.read_csv("./csvs/train.csv")
test = pd.read_csv("./csvs/train.csv")

In [3]:
# Gini coeff
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini


In [30]:
train.head(10)

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,evening_time,night_time,start_session,day_of_week,month,year,yearmonth,session_duration,sites_popularity_coeff,is_short
0,21669,safebrowsing.clients.google.com,2013-01-12 08:05:57,safebrowsing-cache.google.com,2013-01-12 08:05:57,empty,NaN,empty,NaN,empty,...,0,0,8,5,1,2013,201301,0.000000,2.948007,1
1,54843,safebrowsing.clients.google.com,2013-01-12 08:37:23,safebrowsing-cache.google.com,2013-01-12 08:37:23,safebrowsing.clients.google.com,2013-01-12 09:07:07,safebrowsing-cache.google.com,2013-01-12 09:07:09,empty,...,0,0,8,5,1,2013,201301,7.488294,2.948007,1
2,77292,www.apache.org,2013-01-12 08:50:13,www.apache.org,2013-01-12 08:50:14,download.eclipse.org,2013-01-12 08:50:15,www.apache.org,2013-01-12 08:50:15,www.apache.org,...,0,0,8,5,1,2013,201301,1.609438,0.016490,0
3,114021,www.webtide.com,2013-01-12 08:50:17,download.oracle.com,2013-01-12 08:50:17,www.caucho.com,2013-01-12 08:50:18,download.oracle.com,2013-01-12 08:50:18,www.webtide.com,...,0,0,8,5,1,2013,201301,1.386294,0.016490,0
4,146670,public.dhe.ibm.com,2013-01-12 08:50:20,jope.ow2.org,2013-01-12 08:50:20,download.oracle.com,2013-01-12 08:50:20,public.dhe.ibm.com,2013-01-12 08:50:21,jope.ow2.org,...,0,0,8,5,1,2013,201301,1.098612,0.008101,0
5,242171,master.dl.sourceforge.net,2013-01-12 08:50:22,public.dhe.ibm.com,2013-01-12 08:50:23,download.jboss.org,2013-01-12 08:50:23,www.apache.org,2013-01-12 08:50:23,public.dhe.ibm.com,...,0,0,8,5,1,2013,201301,1.386294,0.008101,0
6,57157,download.jboss.org,2013-01-12 08:50:25,public.dhe.ibm.com,2013-01-12 08:50:26,www.apache.org,2013-01-12 08:50:26,download.jboss.org,2013-01-12 08:50:26,dfn.dl.sourceforge.net,...,0,0,8,5,1,2013,201301,1.386294,0.001707,0
7,240201,www.apache.org,2013-01-12 08:50:28,public.dhe.ibm.com,2013-01-12 08:50:28,heanet.dl.sourceforge.net,2013-01-12 08:50:28,download.jboss.org,2013-01-12 08:50:29,www.apache.org,...,0,0,8,5,1,2013,201301,1.386294,0.001635,0
8,210686,www.apache.org,2013-01-12 08:50:31,kent.dl.sourceforge.net,2013-01-12 08:50:32,www.apache.org,2013-01-12 08:50:32,www.apache.org,2013-01-12 08:50:33,dfn.dl.sourceforge.net,...,0,0,8,5,1,2013,201301,1.791759,0.066223,0
9,98804,download.oracle.com,2013-01-12 08:50:37,www.apache.org,2013-01-12 08:50:37,download.oracle.com,2013-01-12 08:50:38,javadl-esd-secure.oracle.com,2013-01-12 08:50:49,gtssl-ocsp.geotrust.com,...,0,0,8,5,1,2013,201301,3.332205,0.867386,0


In [31]:
test.head(5)

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,evening_time,night_time,start_session,day_of_week,month,year,yearmonth,session_duration,sites_popularity_coeff,is_short
0,21669,safebrowsing.clients.google.com,2013-01-12 08:05:57,safebrowsing-cache.google.com,2013-01-12 08:05:57,empty,NaN,empty,NaN,empty,...,0,0,8,5,1,2013,201301,0.000000,2.948007,1
1,54843,safebrowsing.clients.google.com,2013-01-12 08:37:23,safebrowsing-cache.google.com,2013-01-12 08:37:23,safebrowsing.clients.google.com,2013-01-12 09:07:07,safebrowsing-cache.google.com,2013-01-12 09:07:09,empty,...,0,0,8,5,1,2013,201301,7.488294,2.948007,1
2,77292,www.apache.org,2013-01-12 08:50:13,www.apache.org,2013-01-12 08:50:14,download.eclipse.org,2013-01-12 08:50:15,www.apache.org,2013-01-12 08:50:15,www.apache.org,...,0,0,8,5,1,2013,201301,1.609438,0.016490,0
3,114021,www.webtide.com,2013-01-12 08:50:17,download.oracle.com,2013-01-12 08:50:17,www.caucho.com,2013-01-12 08:50:18,download.oracle.com,2013-01-12 08:50:18,www.webtide.com,...,0,0,8,5,1,2013,201301,1.386294,0.016490,0
4,146670,public.dhe.ibm.com,2013-01-12 08:50:20,jope.ow2.org,2013-01-12 08:50:20,download.oracle.com,2013-01-12 08:50:20,public.dhe.ibm.com,2013-01-12 08:50:21,jope.ow2.org,...,0,0,8,5,1,2013,201301,1.098612,0.008101,0


In [49]:
for i in range(1, 11):
    train[f"site{i}"] = train[f"site{i}"].astype("str")
    test[f"site{i}"] = test[f"site{i}"].astype("str")

In [52]:
train = train.drop([f"time{i}" for i in range(1, 11)], axis=1)
test = test.drop([f"time{i}" for i in range(1, 11)], axis=1)

In [53]:
data = pd.concat([train, test])

In [96]:
data

,session_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,...,evening_time,night_time,start_session,day_of_week,month,year,yearmonth,session_duration,sites_popularity_coeff,is_short
0,21669,56,55,0,0,0,0,0,0,0,...,0,0,8,5,1,2013,201301,0,2,1
1,54843,56,55,56,55,0,0,0,0,0,...,0,0,8,5,1,2013,201301,7,2,1
2,77292,946,946,951,946,946,945,948,784,949,...,0,0,8,5,1,2013,201301,1,0,0
3,114021,945,948,949,948,945,946,947,945,946,...,0,0,8,5,1,2013,201301,1,0,0
4,146670,947,950,948,947,950,952,946,951,946,...,0,0,8,5,1,2013,201301,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253528,12224,50,50,48,49,48,52,52,49,303,...,1,0,23,2,4,2014,201404,2,0,0
253529,164438,4207,753,753,52,50,4207,3346,3359,3346,...,1,0,23,2,4,2014,201404,5,0,0
253530,12221,52,3346,784,784,3346,979,3324,7330,3594,...,1,0,23,2,4,2014,201404,3,0,0
253531,156968,3328,3324,3599,3413,753,3328,3599,3359,3359,...,1,0,23,2,4,2014,201404,4,0,0


In [83]:
data["session_duration"] = data["session_duration"].astype(np.int64)
data["sites_popularity_coeff"] = data["sites_popularity_coeff"].astype(np.int64)

In [84]:
data.dtypes

session_id                int64
site1                     int64
site2                     int64
site3                     int64
site4                     int64
site5                     int64
site6                     int64
site7                     int64
site8                     int64
site9                     int64
site10                    int64
target                    int64
morning_time              int64
day_time                  int64
evening_time              int64
night_time                int64
start_session             int64
day_of_week               int64
month                     int64
year                      int64
yearmonth                 int64
session_duration          int64
sites_popularity_coeff    int64
is_short                  int64
dtype: object

In [101]:
y_train

30919     0
94132     0
102102    0
38534     0
160628    0
         ..
185574    0
117952    0
182296    0
52178     0
207951    0
Name: target, Length: 405652, dtype: int64

In [74]:
len(data.columns.to_list())

24

In [58]:
with open("./data/site.pkl", "rb") as fd:
    sites_mapping = pickle.load(fd)

In [61]:
for site in [f"site{i}" for i in range(1, 11)]:
    data[site] = data[site].map(sites_mapping)

In [85]:
data[[f"site{i}" for i in range(1, 11)]] = data[[f"site{i}" for i in range(1, 11)]].fillna(0).astype('int')

In [86]:
cat_features = [i for i in range(1, 22)]

In [87]:
from sklearn.model_selection import train_test_split

X = data.drop("target", axis=1)
y = data["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [95]:
clf = CatBoostClassifier(
    iterations=10, 
    learning_rate=0.1, 

)


clf.fit(X_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_val, y_val), 
        verbose=True
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

from sklearn.metrics import accuracy_score

print('the test accuracy is :{:.6f}'.format(accuracy_score(y_val, clf.predict(X_val))))


0:	learn: 0.4880844	test: 0.4879324	best: 0.4879324 (0)	total: 225ms	remaining: 2.02s
1:	learn: 0.3189793	test: 0.3184227	best: 0.3184227 (1)	total: 473ms	remaining: 1.89s
2:	learn: 0.2200804	test: 0.2193697	best: 0.2193697 (2)	total: 691ms	remaining: 1.61s
3:	learn: 0.1594461	test: 0.1586898	best: 0.1586898 (3)	total: 969ms	remaining: 1.45s
4:	learn: 0.1182906	test: 0.1176226	best: 0.1176226 (4)	total: 1.19s	remaining: 1.19s
5:	learn: 0.0941131	test: 0.0934475	best: 0.0934475 (5)	total: 1.47s	remaining: 979ms
6:	learn: 0.0759971	test: 0.0754520	best: 0.0754520 (6)	total: 1.73s	remaining: 742ms
7:	learn: 0.0639192	test: 0.0634179	best: 0.0634179 (7)	total: 2.02s	remaining: 506ms
8:	learn: 0.0561259	test: 0.0556662	best: 0.0556662 (8)	total: 2.32s	remaining: 258ms
9:	learn: 0.0512568	test: 0.0508487	best: 0.0508487 (9)	total: 2.67s	remaining: 0us

bestTest = 0.05084869193
bestIteration = 9

CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 10, 'learning_rate': 0.1